In [1]:
import numpy as np
import matplotlib.pyplot as plt
import awkward as ak
from coffea import hist, processor
import coffea
import uproot
import sys
sys.path.insert(0,"../")
# from corrections import add_pileup_weight

import pickle
import glob

/uscms/home/kkwok/work/LLP/CMSSW_10_6_20/src/llp_analyzer/coffeaenv/lib/python3.7/site-packages/coffea/util.py:154: FutureWarning: In coffea version v0.8.0 (target date: 31 Dec 2022), this will be an error.
(Set coffea.deprecations_as_errors = True to get a stack trace now.)
ImportError: coffea.hist is deprecated
  warnings.warn(message, FutureWarning)


In [2]:
def rename(name):
    if "HNL" in name:
        m = name.split("_")[2].replace("mHNL","").replace("p0","GeV")
        if "rwctau" in name:
            ct = (name.split("_")[-1].replace("rwctau",""))                
        else:
            ct = (name.split("_")[-1].replace("pl",""))
        label = "_".join([m,ct+"mm"])
    elif "WJet" in name:
        label= "WJetsToLNu"
    elif "EGamma" in name:
        label = "EGamma"
    return label
    

In [20]:
import HNLprocessor.corrections as corrections

xsections = corrections.load_xsection()

# with open('../HNL_histograms_Sept6_ele_all_v2.pickle','rb') as f:        
# with open('../HNL_histograms_Nov18_ele_signal.pickle','rb') as f:            
# with open('../HNL_histograms_Dec6_ele_signals.pickle','rb') as f:              
with open('../HNL_histograms_Dec6_ele_signals.pickle','rb') as f:                  
# with open('../HNL_histograms_Nov17_ele_all.pickle','rb') as f:        
    out_e = pickle.load(f)
# with open('../HNL_histograms_Sept6_muons_all_v2.pickle','rb') as f:
# with open('../HNL_histograms_Nov18_muons_signal.pickle','rb') as f:
# with open('../HNL_histograms_Dec6_muons_signals_2017.pickle','rb') as f:    
with open('../HNL_histograms_Dec6_muons_signals.pickle','rb') as f:    
# with open('../HNL_histograms_Feb23_muons_signal.pickle','rb') as f:        
# with open('../HNL_histograms_Nov17_muons_all.pickle','rb') as f:
    out_mu = pickle.load(f)
lumi = 120 ## fb
# lumi = 1./1000 ## fb
# lumi = 0.939 ## fb
    
for k,h in out_e.items():
    if (type(h)!=hist.Hist): continue
    h.scale({ d: lumi for d in h.identifiers("dataset") if d.name in xsections.keys()}, axis="dataset")
for k,h in out_mu.items():
    if (type(h)!=hist.Hist): continue
    h.scale({ d: lumi for d in h.identifiers("dataset") if d.name in xsections.keys()}, axis="dataset")

In [4]:
out_mu

{'cutflow': <Hist (dataset,region,cutflow) instance at 0x7faa1aca1610>,
 'nCluster': <Hist (dataset,region,nCluster) instance at 0x7faa1aab06d0>,
 'nCluster_dt': <Hist (dataset,region,nCluster) instance at 0x7faa1aab0a50>,
 'nCluster_n-1': <Hist (dataset,nCluster) instance at 0x7faa1aab0b10>,
 'accept': <Hist (dataset,gLLP_csc,gLLP_dt) instance at 0x7faa1aab0bd0>,
 'ClusterSize': <Hist (dataset,region,ClusterSize) instance at 0x7faa1aab0dd0>,
 'ClusterTime': <Hist (dataset,region,ClusterTime) instance at 0x7faa1aab0ed0>,
 'ClusterEta': <Hist (dataset,region,ClusterEta) instance at 0x7faa1aab0f90>,
 'ClusterAvgStation10': <Hist (dataset,region,ClusterAvgStation10) instance at 0x7faa1aaad050>,
 'ClusterNStation10': <Hist (dataset,region,ClusterNStation10) instance at 0x7faa1aaad0d0>,
 'ClusterME11_12': <Hist (dataset,region,ClusterME11_12) instance at 0x7faa1aaad190>,
 'ClusterTimeSpread': <Hist (dataset,region,ClusterTimeSpread) instance at 0x7faa1aaad250>,
 'ClusterJetVetoPt': <Hist (d

In [10]:
h = out_mu['cutflow']
dataList = [a.name for a in h.identifiers("dataset") if "_201" in a.name]
print(dataList)

[]


In [16]:
for d,v in out["accept"].project("gLLP_csc","dataset")[datasets].values('true').items():
    print(d,v[0],v[1])
#     print(d,v[0][1]/v[0].sum())

/uscms/home/kkwok/work/LLP/CMSSW_10_6_20/src/llp_analyzer/coffeaenv/lib/python3.7/site-packages/coffea/hist/hist_tools.py:377: RuntimeWarning: Not all requested indices present in <Cat (name=dataset) instance at 0x7f6b78d4e2d0>
  "Not all requested indices present in %r" % self, RuntimeWarning


In [28]:
out['cutflow'].identifiers("region")

[<StringBin (csc_cutflow) instance at 0x7fb01ef3a690>,
 <StringBin (dt_cutflow) instance at 0x7fb01ef3a710>]

# Cutflow tables functions

In [5]:
from sigfig import round

def getCutFlowArray(h,dataset):
    h_vals = h.integrate("dataset",dataset).values()
    data = np.array([round(float(vals),sigfigs=3) for cut,vals in h_vals.items()])  
    return data
def getCutNames(h,dataset):
    h_vals = h.integrate("dataset",dataset).values()    
    cutnames = [cut[0] for cut,vals in h_vals.items() ]
    return cutnames
def getRelArr(arr,norm=None):
    if norm==None:
        arr_rel = arr[1:]/arr[0]
    else:
        arr_rel = arr[1:]/norm
    arr_rel = ["%.2e"%i for i in arr_rel]    
    return  np.concatenate((["-"],arr_rel))
def getCutEff(arr,norm=None):
    arr_rel = arr[1:]/arr[:-1]
    arr_rel = ["%.3f "%i for i in arr_rel]    
    return  np.concatenate((["-"],arr_rel))
def getSoverB(arr_s,arr_b):
    arr = np.array(arr_s[1:],dtype=float)/np.array(arr_b[1:],dtype=float)
    arr = ['%.2f'%i for i in arr]
    return np.concatenate((["-"],arr))

# Cutflow tables (CSC)

In [31]:
import pandas as pd
h = out_mu['cutflow'].integrate("region",'csc_cutflow')
name= "(4GeV,1m)"
signal = "HNL_muonType_mHNL4p0_pl1000"

# h = out_e['cutflow'].integrate("region",'csc_cutflow')
# name= "(4GeV,1m)"
# signal = "HNL_electronType_mHNL4p0_pl1000"

dataList = [a.name for a in h.identifiers("dataset") if "_201" in a.name]
# print(dataList)

table={}
cutnames          = getCutNames(h,dataList)

table["data"]     =getCutFlowArray(h,dataList)
table["data_cul"] =getRelArr(table['data'])
table["data_rel"] =getCutEff(table['data'])

table[name]=getCutFlowArray(h,signal)[1:]
table[name+"cul"] =getRelArr(table[name])
table[name+"rel"] =getCutEff(table[name])
table[name+"S/B"] =getSoverB(table[name+"rel"],table["data_rel"])

df_abs = pd.DataFrame.from_dict(table,orient="index",columns=cutnames,dtype=str).transpose()


In [33]:
df_abs.to_latex

<bound method NDFrame.to_latex of                       data  data_cul data_rel (4GeV,1m) (4GeV,1m)cul  \
NoSelection    309000000.0         -        -      81.8            -   
trigger_mu     250000000.0  8.09e-01   0.809       40.7     4.98e-01   
MET            157000000.0  5.08e-01   0.628       28.5     3.48e-01   
METfilters     157000000.0  5.08e-01   1.000       28.5     3.48e-01   
good_lepton    102000000.0  3.30e-01   0.650       26.2     3.20e-01   
n_cls           61200000.0  1.98e-01   0.600       13.3     1.63e-01   
dr_lep          17700000.0  5.73e-02   0.289       12.6     1.54e-01   
ME11_12_veto     1190000.0  3.85e-03   0.067       8.64     1.06e-01   
jetVeto          1070000.0  3.46e-03   0.899       7.93     9.69e-02   
muonVeto         1060000.0  3.43e-03   0.991        7.6     9.29e-02   
MB1seg_veto      1060000.0  3.43e-03   1.000       7.51     9.18e-02   
RB1_veto         1050000.0  3.40e-03   0.991       7.37     9.01e-02   
RE12_veto        1040000.0  3.

In [32]:
df_abs

,data,data_cul,data_rel,"(4GeV,1m)","(4GeV,1m)cul","(4GeV,1m)rel","(4GeV,1m)S/B"
NoSelection,309000000.0,-,-,81.8,-,-,-
trigger_mu,250000000.0,8.09e-01,0.809,40.7,4.98e-01,0.498,0.62
MET,157000000.0,5.08e-01,0.628,28.5,3.48e-01,0.700,1.11
METfilters,157000000.0,5.08e-01,1.000,28.5,3.48e-01,1.000,1.00
good_lepton,102000000.0,3.30e-01,0.650,26.2,3.20e-01,0.919,1.41
n_cls,61200000.0,1.98e-01,0.600,13.3,1.63e-01,0.508,0.85
dr_lep,17700000.0,5.73e-02,0.289,12.6,1.54e-01,0.947,3.28
ME11_12_veto,1190000.0,3.85e-03,0.067,8.64,1.06e-01,0.686,10.24
jetVeto,1070000.0,3.46e-03,0.899,7.93,9.69e-02,0.918,1.02
muonVeto,1060000.0,3.43e-03,0.991,7.6,9.29e-02,0.958,0.97


# Cutflow tables (DT)


In [16]:
table

{}

In [14]:
import pandas as pd
h = out_mu['cutflow'].integrate("region",'dt_cutflow')
name= "(4GeV,1m)"
signal = "HNL_muonType_mHNL4p0_pl1000"

# h = out_e['cutflow'].integrate("region",'dt_cutflow')
# name= "(4GeV,1m)"
# signal = "HNL_electronType_mHNL4p0_pl1000"

dataList = [a.name for a in h.identifiers("dataset") if "_201" in a.name]
print(dataList)

table={}
cutnames          = getCutNames(h,dataList)
cutnames[0]="NoSelection"
table["data"]     =getCutFlowArray(h,dataList)
table["data_cul"] =getRelArr(table['data'])
table["data_rel"] =getCutEff(table['data'])
# name = "Wjet"
# table[name]=getCutFlowArray(h,"WJetsToLNu_TuneCP5_13TeV-madgraphMLM-pythia8")[1:]
# table[name+"rel"] =getCutEff(table[name])

name= "(4GeV,1m)"
table[name]=getCutFlowArray(h,signal)[1:]
table[name+"cul"] =getRelArr(table[name])
table[name+"rel"] =getCutEff(table[name])
table[name+"S/B"] =getSoverB(table[name+"rel"],table["data_rel"])

df_abs = pd.DataFrame.from_dict(table,orient="index",columns=cutnames,dtype=str).transpose()


[]


IndexError: list assignment index out of range

In [ ]:
df_abs

# Cut flow table signal

In [ ]:
#    {'HNL_muonType_mHNL1p0_pl10000': 496976.0,
#      'HNL_muonType_mHNL2p0_pl10000_rwctau5000': 444704.0,
#      'HNL_muonType_mHNL4p0_pl1000': 659711.0,
#      'HNL_muonType_mHNL4p0_pl100_rwctau80': 662653.0,
#      'HNL_muonType_mHNL1p0_pl10000_rwctau20000': 496976.0,

In [21]:
h.values()

dict_values([635215.0, 634869.0, 634869.0, 496811.0, 496740.0, 496811.0, 496811.0, 496811.0, 498926.0, 496811.0, 496811.0, 498244.0, 498926.0, 498926.0, 498926.0, 498926.0, 498926.0, 636184.0, 498926.0, 498926.0, 636184.0, 633118.0, 636184.0, 636184.0, 633118.0, 633118.0, 633118.0, 633118.0, 633118.0, 498149.0, 636184.0, 685947.0, 498149.0, 634681.0, 498149.0, 498149.0, 685947.0, 5867335.0, 496976.0, 496976.0, 496976.0, 634869.0])

In [22]:
import pandas as pd
h = out_mu['cutflow'].integrate("region",'csc_cutflow')

h_e = out_e['cutflow'].integrate("region",'csc_cutflow')

table={}

name= "(4GeV,1m) ele "
signal = "HNL_electronType_mHNL4p0_pl1000"
cutnames = getCutNames(h_e,signal)[1:]

table[name]=getCutFlowArray(h_e,signal)[1:]
table[name+"rel"] =getCutEff(table[name])

name= "(4GeV,1m) mu "
signal = "HNL_muonType_mHNL4p0_pl1000"

table[name]=getCutFlowArray(h,signal)[1:]
table[name+"rel"] =getCutEff(table[name])

df_abs = pd.DataFrame.from_dict(table,orient="index",columns=cutnames,dtype=str).transpose()


In [23]:
df_abs

,"(4GeV,1m) ele","(4GeV,1m) ele rel","(4GeV,1m) mu","(4GeV,1m) mu rel"
Acceptance_csc,89.0,-,81.7,-
trigger_ele,26.5,0.298,40.7,0.498
MET,19.2,0.725,28.5,0.700
METfilters,19.2,1.000,28.5,1.000
good_lepton,11.5,0.599,26.2,0.919
n_cls,5.21,0.453,13.3,0.508
dr_lep,5.18,0.994,12.6,0.947
ME11_12_veto,3.66,0.707,8.6,0.683
jetVeto,3.36,0.918,7.87,0.915
muonVeto,3.26,0.970,7.56,0.961


In [38]:
import pandas as pd
h = out_mu['cutflow'].integrate("region",'dt_cutflow')

# h = out_e['cutflow'].integrate("region",'csc_cutflow')

table={}

name= "(4GeV,1m)"
signal = "HNL_muonType_mHNL4p0_pl1000"
cutnames = getCutNames(h,signal)

name= "(1GeV,1m)"
signal = "HNL_muonType_mHNL1p0_pl1000"
table[name]=getCutFlowArray(h,signal)
table[name+"rel"] =getCutEff(table[name])
name= "(2GeV,1m)"
signal = "HNL_muonType_mHNL2p0_pl1000"
table[name]=getCutFlowArray(h,signal)
table[name+"rel"] =getCutEff(table[name])

name= "(4GeV,1m)"
signal = "HNL_muonType_mHNL4p0_pl1000"
table[name]=getCutFlowArray(h,signal)
table[name+"rel"] =getCutEff(table[name])

df_abs = pd.DataFrame.from_dict(table,orient="index",columns=cutnames,dtype=str).transpose()


In [39]:
df_abs

,"(1GeV,1m)","(1GeV,1m)rel","(2GeV,1m)","(2GeV,1m)rel","(4GeV,1m)","(4GeV,1m)rel"
NoSelection,1450000.0,-,45300.0,-,1380.0,-
Acceptance_dt,44200.0,0.030,2180.0,0.048,87.5,0.063
trigger_mu,24500.0,0.554,1280.0,0.587,55.3,0.632
MET,16500.0,0.673,871.0,0.680,38.3,0.693
METfilters,16500.0,1.000,869.0,0.998,38.2,0.997
good_lepton,14100.0,0.855,747.0,0.860,32.8,0.859
n_cls_dt,4710.0,0.334,266.0,0.356,14.1,0.430
dr_lep,4190.0,0.890,237.0,0.891,12.9,0.915
dt_MB1veto,2430.0,0.580,136.0,0.574,6.91,0.536
dt_jetVeto,2360.0,0.971,132.0,0.971,6.7,0.970


In [79]:
import pandas as pd
h = out_e['cutflow'].integrate("region",'dt_cutflow')


table={}

name= "(4GeV,1m)"
signal = "HNL_electronType_mHNL4p0_pl1000"
cutnames = getCutNames(h,signal)

name= "(1GeV,1m)"
signal = "HNL_electronType_mHNL1p0_pl1000"
table[name]=getCutFlowArray(h,signal)
table[name+"rel"] =getCutEff(table[name])
name= "(2GeV,1m)"
signal = "HNL_electronType_mHNL2p0_pl1000"
table[name]=getCutFlowArray(h,signal)
table[name+"rel"] =getCutEff(table[name])

name= "(4GeV,1m)"
signal = "HNL_electronType_mHNL4p0_pl1000"
table[name]=getCutFlowArray(h,signal)
table[name+"rel"] =getCutEff(table[name])

df_abs = pd.DataFrame.from_dict(table,orient="index",columns=cutnames,dtype=str).transpose()


In [80]:
df_abs.to_excel

<bound method NDFrame.to_excel of                (1GeV,1m) (1GeV,1m)rel (2GeV,1m) (2GeV,1m)rel (4GeV,1m)  \
NoSelection    1590000.0            -   49700.0            -    1510.0   
Acceptance_dt    47400.0       0.030     2370.0       0.048       93.9   
trigger_ele      15700.0       0.331      821.0       0.346       34.6   
MET              10900.0       0.694      570.0       0.694       24.5   
METfilters       10900.0       1.000      570.0       1.000       24.5   
good_lepton       6910.0       0.634      351.0       0.616       15.2   
n_cls_dt          1860.0       0.269      101.0       0.288       5.67   
dr_lep            1850.0       0.995      101.0       1.000       5.64   
dt_MB1veto        1140.0       0.616       56.9       0.563       3.06   
dt_jetVeto        1110.0       0.974       55.7       0.979       2.99   
dt_muonVeto       1110.0       1.000       55.6       0.998       2.99   
dt_RPC            1100.0       0.991       54.8       0.986       2.95   
dt_M

# OLD CODE Below

In [14]:
from sigfig import round
h = out['cutflow'].integrate("region",'csc_cutflow')

datasets = [
#  'HNL_muonType_mHNL4p0_pl1000',
    "HNL_electronType_mHNL4p0_pl1000"
]
table = {}
table_rel = {}
for i,d in enumerate(np.repeat(datasets,2)): 
    h_vals = h.integrate("dataset",d).values()
    cutnames = [cut[0] for cut,vals in h_vals.items()]
    data = np.array([vals for cut,vals in h_vals.items()])
    label= rename(d)    
    acc = data[1]
    data[1] = data[1]/data[0]
    ## even column: divide by acceptance
    if i%2==0:
        table_val = np.insert(data[2:]/acc, 0,data[1])*100
        table[label+"_acc"] = [round(i,sigfigs=3) for i in table_val]               
    else:
    ## odd column: divide by last cut
        table_val = np.insert(data[3:]/data[2:-1], 0,data[2]/acc) #
        table_val = np.insert(table_val, 0,data[1])
        table_val = table_val*100
        table[label+"_rel"] = [round(i,sigfigs=3) for i in table_val]       
#     table[label]= np.round(data)
#     table_rel[label] = np.round(data[1]+data[1:]/data[2:,5)*100    
# print(table)
import pandas as pd
df = pd.DataFrame.from_dict(table,orient="index",columns=cutnames[1:],dtype=float).transpose()

In [22]:
h.integrate("dataset",dataList).values()

{('NoSelection',): array(56294301.),
 ('trigger_ele',): array(24712128.),
 ('MET',): array(16291161.),
 ('METfilters',): array(16270891.),
 ('good_lepton',): array(5182752.),
 ('n_cls_dt',): array(836129.),
 ('dt_MB1veto',): array(20693.),
 ('dt_jetVeto',): array(14662.),
 ('dt_muonVeto',): array(12284.),
 ('dt_RPC',): array(8902.),
 ('dt_MB1adj',): array(8220.),
 ('dt_time',): array(1972.),
 ('dt_deadzones',): array(1714.)}

In [15]:
h.integrate("dataset","HNL_electronType_mHNL4p0_pl1000").values()

{('NoSelection',): array(1765.83852345),
 ('Acceptance_csc',): array(103.86633914),
 ('trigger_ele',): array(30.64992369),
 ('MET',): array(22.24596668),
 ('METfilters',): array(22.24400299),
 ('good_lepton',): array(13.16019213),
 ('n_cls',): array(5.95459364),
 ('ME11_12_veto',): array(4.21632807),
 ('jetVeto',): array(3.88459938),
 ('muonVeto',): array(3.78423397),
 ('MB1seg_veto',): array(3.72953528),
 ('RB1_veto',): array(3.66531954),
 ('IntimeCut',): array(3.61891698),
 ('timeSpreadCut',): array(3.54989064),
 ('ClusterID',): array(2.06989819)}

In [20]:
from sigfig import round
h = out['cutflow'].integrate("region",'dt_cutflow')

datasets = [
 dataList,dataList,dataList
]
table = {}
table_rel = {}
# for i,d in enumerate(np.repeat(datasets,2)): 
for i,d in enumerate(datasets):     
    h_vals = h.integrate("dataset",d).values()
    cutnames = [cut[0] for cut,vals in h_vals.items()]
    data = np.array([vals for cut,vals in h_vals.items()])
    label= "ele"   
    ## even column: abs.
    if i%3==0:    
#         table_val = data[1:]
#         table[label+"_raw"] = [round(i,sigfigs=3) for i in table_val]                       
        table_val = data[1:]/data[0]*100        
        table[label+"_abs"] = [round(i,sigfigs=3) for i in table_val]                       
    if i%3==1:
        table_val = data[1:]/data[0]*100        
        table[label+"_abs"] = [round(i,sigfigs=3) for i in table_val]               
    elif i%3==2:
        table_val = data[1:]/data[0:-1] #
        table_val = table_val*100
        table[label+"_rel"] = [round(i,sigfigs=3) for i in table_val]        
import pandas as pd
df = pd.DataFrame.from_dict(table,orient="index",columns=cutnames[1:],dtype=float).transpose()

In [21]:
df

,ele_abs,ele_rel
trigger_ele,43.90000,43.90
MET,28.90000,65.90
METfilters,28.90000,99.90
good_lepton,9.21000,31.90
n_cls_dt,1.49000,16.10
dt_MB1veto,0.03680,2.47
dt_jetVeto,0.02600,70.90
dt_muonVeto,0.02180,83.80
dt_RPC,0.01580,72.50
dt_MB1adj,0.01460,92.30


In [349]:
df_cum

,1GeV_100mm,1GeV_10mm,2GeV_10mm,4GeV_1000mm,EGamma,WJetsToLNu
trigger,24.0589,24.0109,24.0328,23.8570,54.1996,5.2042
Nlep>=1,13.7058,13.6865,13.7120,13.4945,13.8676,2.9073
MET,8.4654,6.6268,6.5854,9.1336,9.5585,2.1811
METfilters,8.3017,6.5969,6.5799,9.0615,9.5537,2.1806
n_cls,0.8020,0.3477,0.2794,0.7005,8.2525,2.1024
ME11_12,0.3172,0.0424,0.0299,0.2807,0.5281,0.2657
JetVeto,0.2870,0.0388,0.0260,0.2575,0.4734,0.2423
MuVeto,0.2773,0.0384,0.0260,0.2516,0.4719,0.2418
MB1seg,0.2687,0.0382,0.0260,0.2454,0.4696,0.2408
RB1,0.2646,0.0381,0.0260,0.2411,0.4677,0.2396


# N-1 cut flow

In [206]:
h = out['nCluster_n-1']
h = h.project('Nminus1',"dataset")

h.label="Efficiency"
norm = {}

table = {}
table2= {}
datasets = [
    "HNL_electronType_mHNL1p0_pl10",
#     "HNL_electronType_mHNL1p0_pl100",
#     "HNL_electronType_mHNL1p0_pl1000",    
    "HNL_electronType_mHNL2p0_pl10",
#     "HNL_electronType_mHNL2p0_pl1000",
    "HNL_electronType_mHNL4p0_pl100",
#     "HNL_electronType_mHNL4p0_pl1000",  
    "WJetsToLNu_TuneCP5_13TeV-madgraphMLM-pythia8",
    "EGamma_2018A"
]
for d,cutflow in h[datasets].values().items():
    name = d[0]
    label=rename(name)
    data = np.round(cutflow/cutflow[1]*100,3)
    data = data[data!=0.0]    
#     print(label,data)
    table[label] = data
    data2 = data[2:]-data[-1]
    table2[label] = data2
    
cols = ["Inclusive","Presel","ME11_12","JetVeto","MuVeto","MB1seg","RB1",
                            "Time","TimeSpread","ClusID","all"]
    
df = pd.DataFrame.from_dict(table,orient="index",columns=cols)
df = df.transpose().sort_index(1)


In [212]:
cols = ["ME11_12","JetVeto","MuVeto","MB1seg","RB1","Time","TimeSpread","ClusID","all"]
    
df = pd.DataFrame.from_dict(table2,orient="index",columns=cols)
df = df.transpose().sort_index(1)

In [213]:
df

,1GeV_10mm,2GeV_10mm,4GeV_100mm,EGamma,WJetsToLNu
ME11_12,0.000,0.108,0.549,0.015,0.015
JetVeto,0.043,0.000,0.087,0.001,0.000
MuVeto,1.867,0.649,3.819,0.650,0.616
MB1seg,0.000,0.000,0.000,0.001,0.000
RB1,0.000,0.000,0.029,0.002,0.004
Time,1.390,1.514,0.723,0.281,2.076
TimeSpread,0.043,0.000,0.058,0.029,0.026
ClusID,3.692,1.135,12.963,0.530,0.715
all,0.000,0.000,0.000,0.000,0.000
